In [544]:
# 라이브러리 로드

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from scipy import stats
plt.rc('font', family='Malgun Gothic') 
mpl.rcParams['axes.unicode_minus'] = False
%matplotlib inline
import warnings

from scipy.stats.mstats  import winsorize

In [545]:
# 데이터 로드 

df = pd.read_csv('./datasets/전처리준비/코스닥결측치처리완료.csv',index_col=0)

#  타겟 데이터 형식 바꿔주기
df['target_1']  =df['target_1'].astype(int)
df['target_2']  =df['target_2'].astype(int)
df['target_3']  =df['target_3'].astype(int)


In [546]:
print('전체 행:', df.shape[0],'개')
print('전체 열:', df.shape[1],'개')
print('전체 기업 수',len(df['회사명'].unique()))
print("결측치 수 : ", df.isna().sum().sum())

전체 행: 9574 개
전체 열: 69 개
전체 기업 수 1440
결측치 수 :  0


In [547]:
# 코스피 데이터와 컬럼 순서 맞춰주기

df = df[['회사명', '거래소코드', '회계년도', '자본', '차입금의존도', '유보율', '현금비율', '부채비율', '당좌비율',
       '순운전자본비율', '영업손익', '감가상각비', '산업군', '총자본정상영업이익률', '자기자본순이익률', '총자본순이익률',
       '매출액총이익률', '금융비용부담률', '경영자본순이익률', '자본금회전률', '유형자산회전율', '재고자산회전률',
       '자기자본회전률', '타인자본회전률', '총자본회전률', '이익잉여금', '영업활동으로 인한 현금흐름', '금융수익',
       '법인세비용차감전순이익', '당기순이익', '합계_기말인원(명)', '외국인지분율', '대주주지분율', '단기차입금',
       '유동성장기부채', 'CAPEX', '유형자산증가율', '매출액증가율', '자기자본증가율', '총자본증가율',
       '정상영업이익증가율', '장단기금리차', 'cpi', '업력', '누적수익성비율', '이자부담률', '부채상환계수',
       '현금흐름 대 자산', '현금흐름 대 매출액', '토빈Q', 'TMD', '시총/총자산', '연구개발비', 'RD',
       '직원수변동률%', '직원인건비총액', 'EBIT/총자산', 'PBR', '당기전기영업손익', 'FINDEP',
       'EBITDA마진율', 'PBR변화율', 'ROA변화율', 'target_1', 'target_2', 'target_3',
       '현금흐름/총부채비율', '영업현금흐름-단기차입금', 'abs(영업현금흐름-당기순이익)/매출액']]

In [548]:
# 산식에 이용했거나 

drop_ls = ["자본", "유보율", "영업손익", "감가상각비", "이익잉여금", "영업활동으로 인한 현금흐름", "금융수익", "법인세비용차감전순이익", "당기순이익", "단기차입금", "유동성장기부채", "CAPEX"
 , "cpi", "장단기금리차", "연구개발비",'직원수변동률%', '직원인건비총액', "PBR", "현금비율", "당좌비율", "토빈Q", "RD", "경영자본순이익률", "합계_기말인원(명)"]

df.drop(columns=drop_ls, axis=1 ,inplace=True)

In [549]:
pd.set_option( "display.max.columns", None)
df.select_dtypes("float").describe()

,차입금의존도,부채비율,순운전자본비율,총자본정상영업이익률,자기자본순이익률,총자본순이익률,매출액총이익률,금융비용부담률,자본금회전률,유형자산회전율,재고자산회전률,자기자본회전률,타인자본회전률,총자본회전률,외국인지분율,대주주지분율,유형자산증가율,매출액증가율,자기자본증가율,총자본증가율,정상영업이익증가율,누적수익성비율,이자부담률,부채상환계수,현금흐름 대 자산,현금흐름 대 매출액,TMD,시총/총자산,EBIT/총자산,당기전기영업손익,FINDEP,EBITDA마진율,PBR변화율,ROA변화율,현금흐름/총부채비율,영업현금흐름-단기차입금,abs(영업현금흐름-당기순이익)/매출액
count,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9.574000e+03,9.574000e+03,9574.000000,9574.000000,9574.000000,9574.000000,9.574000e+03,9574.000000
mean,22.058254,121.535895,19.681602,2.073494,-6.432872,-0.935825,22.236496,2.347658,20.403892,13.334268,215.418328,1.729680,2.518764,0.835486,4.872501,36.301811,47.195716,28.768221,13.879221,11.951400,-137.312528,0.101107,190.744452,130.487703,0.033474,-0.237253,0.960585,1.234991,0.000248,8.765474e+03,2.255270e+02,-18.754304,23.346677,-110.149951,0.142729,-1.796421e+04,0.267438
std,19.199957,774.709395,29.924214,10.395685,87.266051,15.997633,20.127054,56.130112,35.770992,78.931957,3789.502651,1.654529,2.014457,0.531200,8.703060,16.413630,765.298651,1525.279600,95.226863,40.641125,6674.252773,0.769592,1901.703965,1687.508819,0.116303,8.881171,2.264870,1.669695,0.185732,3.019600e+04,6.219024e+04,733.672760,492.439239,3430.151927,0.504367,2.447359e+05,2.616358
min,0.000000,0.000000,-1243.210000,-204.740000,-5300.390000,-318.880000,-307.410000,-705.810000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-100.000000,-98.900616,-1079.677183,-95.273372,-532450.000000,-38.243959,-5511.340000,-5511.340000,-3.752854,-583.029412,0.003003,0.012305,-4.745856,-1.665870e+05,-6.259930e+05,-46389.573529,-10065.643303,-252600.000000,-10.044103,-1.405336e+07,0.000004
25%,6.592500,32.990000,2.402500,-1.340000,-7.595000,-3.957500,10.170000,-0.180000,4.850000,1.610000,4.250000,0.750000,1.190000,0.480000,0.672500,24.020000,-5.310925,-9.225646,-1.879913,-3.146914,-58.055115,-0.025189,0.530743,0.039522,-0.014878,-0.020582,0.199013,0.465334,-0.026377,-1.000375e+03,-6.498771e+03,0.829039,-21.350355,-74.642018,-0.032974,-1.851450e+04,0.030848
50%,19.825000,68.760000,20.390000,2.815000,3.580000,1.970000,18.535000,0.510000,11.265000,2.930000,7.505000,1.270000,2.000000,0.740000,1.820000,35.220000,1.280050,4.629435,6.094909,5.506082,2.274377,0.218347,5.143487,0.320278,0.041221,0.051205,0.459023,0.790577,0.028619,2.995750e+03,-7.914500e+00,5.406106,-0.184075,-7.725957,0.098934,-4.369500e+03,0.073235
75%,34.267500,129.217500,38.090000,7.110000,10.190000,6.220000,31.880000,1.770000,23.627500,5.960000,15.225000,2.160000,3.270000,1.080000,4.690000,47.560000,17.151741,19.330522,16.817845,17.346109,66.311495,0.430462,27.010410,1.706270,0.092954,0.124939,1.006815,1.403393,0.071803,1.033712e+04,6.351224e+03,11.832725,30.342740,71.654509,0.287225,4.045250e+03,0.156813
max,749.400000,65433.660000,94.820000,104.810000,348.190000,167.950000,100.000000,4581.790000,1165.270000,4055.070000,195075.540000,36.170000,41.030000,5.420000,84.170000,100.000000,50862.500000,149019.444444,5045.605307,922.620863,126440.000000,3.210667,79184.250000,79184.250000,1.180479,4.927749,72.854765,33.247670,3.066083,1.185388e+06,3.454190e+06,79.961348,45142.470366,146066.666667,4.418856,6.059470e+05,175.940945


In [550]:
df.columns

Index(['회사명', '거래소코드', '회계년도', '차입금의존도', '부채비율', '순운전자본비율', '산업군',
       '총자본정상영업이익률', '자기자본순이익률', '총자본순이익률', '매출액총이익률', '금융비용부담률', '자본금회전률',
       '유형자산회전율', '재고자산회전률', '자기자본회전률', '타인자본회전률', '총자본회전률', '외국인지분율',
       '대주주지분율', '유형자산증가율', '매출액증가율', '자기자본증가율', '총자본증가율', '정상영업이익증가율', '업력',
       '누적수익성비율', '이자부담률', '부채상환계수', '현금흐름 대 자산', '현금흐름 대 매출액', 'TMD',
       '시총/총자산', 'EBIT/총자산', '당기전기영업손익', 'FINDEP', 'EBITDA마진율', 'PBR변화율',
       'ROA변화율', 'target_1', 'target_2', 'target_3', '현금흐름/총부채비율',
       '영업현금흐름-단기차입금', 'abs(영업현금흐름-당기순이익)/매출액'],
      dtype='object')

In [551]:
최초회사 = df['회사명'].unique()
len(최초회사)

1440

In [552]:
# colormap = plt.cm.hot
# 수치형_col = df.select_dtypes(float)

# corr =수치형_col.corr()
# # mask = np.triu(np.ones_like(corr, dtype=np.bool))
# plt.figure(figsize=(30, 30))
# plt.rcParams['axes.unicode_minus'] = False
# plt.rc('font', family="Malgun Gothic")
# plt.title("Correlation of Features", y=1.05, size=15)
# sns.heatmap(corr,  linewidths=0.1, vmax=1.0, vmin=-1.0, square=True,
#             cmap='RdYlBu_r', linecolor="white", annot=True, annot_kws={'size': 10})

In [553]:
print('전체 행:', df.shape[0],'개')
print('전체 열:', df.shape[1],'개')
print('전체 기업 수',len(df['회사명'].unique()))
print("결측치 수 : ", df.isna().sum().sum())

전체 행: 9574 개
전체 열: 45 개
전체 기업 수 1440
결측치 수 :  0


In [554]:
#  수치형 df 만들기
df_float =  df.select_dtypes("float")

In [555]:
pd.set_option( "display.max.columns", None)
df_float.describe()

,차입금의존도,부채비율,순운전자본비율,총자본정상영업이익률,자기자본순이익률,총자본순이익률,매출액총이익률,금융비용부담률,자본금회전률,유형자산회전율,재고자산회전률,자기자본회전률,타인자본회전률,총자본회전률,외국인지분율,대주주지분율,유형자산증가율,매출액증가율,자기자본증가율,총자본증가율,정상영업이익증가율,누적수익성비율,이자부담률,부채상환계수,현금흐름 대 자산,현금흐름 대 매출액,TMD,시총/총자산,EBIT/총자산,당기전기영업손익,FINDEP,EBITDA마진율,PBR변화율,ROA변화율,현금흐름/총부채비율,영업현금흐름-단기차입금,abs(영업현금흐름-당기순이익)/매출액
count,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9574.000000,9.574000e+03,9.574000e+03,9574.000000,9574.000000,9574.000000,9574.000000,9.574000e+03,9574.000000
mean,22.058254,121.535895,19.681602,2.073494,-6.432872,-0.935825,22.236496,2.347658,20.403892,13.334268,215.418328,1.729680,2.518764,0.835486,4.872501,36.301811,47.195716,28.768221,13.879221,11.951400,-137.312528,0.101107,190.744452,130.487703,0.033474,-0.237253,0.960585,1.234991,0.000248,8.765474e+03,2.255270e+02,-18.754304,23.346677,-110.149951,0.142729,-1.796421e+04,0.267438
std,19.199957,774.709395,29.924214,10.395685,87.266051,15.997633,20.127054,56.130112,35.770992,78.931957,3789.502651,1.654529,2.014457,0.531200,8.703060,16.413630,765.298651,1525.279600,95.226863,40.641125,6674.252773,0.769592,1901.703965,1687.508819,0.116303,8.881171,2.264870,1.669695,0.185732,3.019600e+04,6.219024e+04,733.672760,492.439239,3430.151927,0.504367,2.447359e+05,2.616358
min,0.000000,0.000000,-1243.210000,-204.740000,-5300.390000,-318.880000,-307.410000,-705.810000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-100.000000,-98.900616,-1079.677183,-95.273372,-532450.000000,-38.243959,-5511.340000,-5511.340000,-3.752854,-583.029412,0.003003,0.012305,-4.745856,-1.665870e+05,-6.259930e+05,-46389.573529,-10065.643303,-252600.000000,-10.044103,-1.405336e+07,0.000004
25%,6.592500,32.990000,2.402500,-1.340000,-7.595000,-3.957500,10.170000,-0.180000,4.850000,1.610000,4.250000,0.750000,1.190000,0.480000,0.672500,24.020000,-5.310925,-9.225646,-1.879913,-3.146914,-58.055115,-0.025189,0.530743,0.039522,-0.014878,-0.020582,0.199013,0.465334,-0.026377,-1.000375e+03,-6.498771e+03,0.829039,-21.350355,-74.642018,-0.032974,-1.851450e+04,0.030848
50%,19.825000,68.760000,20.390000,2.815000,3.580000,1.970000,18.535000,0.510000,11.265000,2.930000,7.505000,1.270000,2.000000,0.740000,1.820000,35.220000,1.280050,4.629435,6.094909,5.506082,2.274377,0.218347,5.143487,0.320278,0.041221,0.051205,0.459023,0.790577,0.028619,2.995750e+03,-7.914500e+00,5.406106,-0.184075,-7.725957,0.098934,-4.369500e+03,0.073235
75%,34.267500,129.217500,38.090000,7.110000,10.190000,6.220000,31.880000,1.770000,23.627500,5.960000,15.225000,2.160000,3.270000,1.080000,4.690000,47.560000,17.151741,19.330522,16.817845,17.346109,66.311495,0.430462,27.010410,1.706270,0.092954,0.124939,1.006815,1.403393,0.071803,1.033712e+04,6.351224e+03,11.832725,30.342740,71.654509,0.287225,4.045250e+03,0.156813
max,749.400000,65433.660000,94.820000,104.810000,348.190000,167.950000,100.000000,4581.790000,1165.270000,4055.070000,195075.540000,36.170000,41.030000,5.420000,84.170000,100.000000,50862.500000,149019.444444,5045.605307,922.620863,126440.000000,3.210667,79184.250000,79184.250000,1.180479,4.927749,72.854765,33.247670,3.066083,1.185388e+06,3.454190e+06,79.961348,45142.470366,146066.666667,4.418856,6.059470e+05,175.940945


In [556]:
def qqplot(df,ncols,nrows):
    plt.rcParams['axes.unicode_minus'] = False
    plt.rcParams["font.family"] = "Malgun Gothic"
   
    fig, axs = plt.subplots(nrows, ncols, constrained_layout=True)
    fig.set_size_inches((50, 80))

    for ax, i in zip(axs.ravel(), df):
            stats.probplot(df[i], dist=stats.norm, plot=ax)
            ax.set_title(str(i))

    plt.show()

# qqplot(df_float,6,9)


In [557]:
## boxplot

def dist(df_1,target,col_n, row_n):
    plt.rcParams['font.family'] = "Malgun Gothic"
    plt.rcParams['axes.unicode_minus'] = False # 마이너스숫자 출력처리

    fig, ax = plt.subplots(ncols=col_n, nrows=row_n, figsize=(20, row_n*5))

    for i, col in enumerate(df_1.columns[:]):
        sns.boxplot(y=col,x=target, ax=ax[int(i/col_n), int(i%col_n)],data=df_1) # df_1의 데이터를 빨간색으로 박스플롯 그리기
       
# dist(df_float,df["target_2"],6,9)
# plt.show() # 박스플롯 출력|

In [558]:
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))
print("결측치 수 : ", df.isna().sum().sum())

전체 행: 9574 개
전체 기업 수 1440
결측치 수 :  0


---
## 이상치 처리

In [559]:
# #  데이터를 하나하나 봐서 이상치 제거


# # (주)베노홀딩스, 2017, 현금흐름/총부채비율
# # 대한그린파워(주), 2015, 현금흐름/총부채비율
# # (주)레고켐바이오사이언스, 2014, 현금흐름 대 매출액
# # (주)스튜디오산타클로스엔터테인먼트, 2016, 현금비율
# # (주)희림종합건축사사무소, 2017, 현금비율
# # (주)알테오젠, 2015, 현금비율
# # (주)엔케이맥스, 2016, 현금비율
# # (주)정상제이엘에스, 2016, 타인자본회전률
# # (주)휴먼엔, 2017, 타인자본회전률
# # 대한그린파워(주), 2014, 총자본회전률
# # 대한그린파워(주), 2016, 총자본회전률
# # (주)에이프로젠헬스케어, 2017, 총자본증가율
# # 한국전자금융(주), 2017, 총자본증가율
# # (주)아난티, 2015, 총자본증가율
# # (주)헥토이노베이션, 2016, 총자본증가율
# # # 대한그린파워(주), 2015, 총자본정상영업이익률
# # (주)제이테크놀로지, 2012, 전기당기영업손익
# # (주)상지카일룸, 2017, 자기자본증가율
# # (주)동원개발, 2016, 이자부담률
# # (주)동서, 2015, 이자부담률
# # (주)웹젠, 2015, 이자부담률
# # (주)지에스홈쇼핑, 2015, 이자부담률
# # (주)위메이드, 2013, 연구개발비
# # (주)케이피엠테크, 2012, 부채비율
# # 아이이(주), 2013, 매출액증가율
# # (주)아이디스홀딩스, 2013, 매출액증가율
# # (주)와이비엠넷, 2013, 금융비용부담률
# # 크리스탈지노믹스(주), 2013, 금융비용부담률
# # 아이진(주), 2016, 금융비용부담률
# # (주)파라다이스, 2014, Findef
# # (주)셀트리온, 2014, Ebitda

# # *(영업현금흐름-당기순이익)/매출액(절대값) —> 수정해야해서 서치 안함.

# # 위의 기업 리스트 제거

# df.drop(index = df[(df["회사명"]== "(주)베노홀딩스") & (df["회계년도"] == "2017")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "대한그린파워(주)") & (df["회계년도"] == "2015")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "(주)레고켐바이오사이언스") & (df["회계년도"] == "2014")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "(주)스튜디오산타클로스엔터테인먼트") & (df["회계년도"] == "2016")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "(주)희림종합건축사사무소") & (df["회계년도"] == "2016")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "(주)알테오젠") & (df["회계년도"] == "2015")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "엔케이맥스") & (df["회계년도"] == "2016")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "(주)정상제이엘에스") & (df["회계년도"] == "2016")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "휴먼엔") & (df["회계년도"] == "2017")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "대한그린파워(주)") & (df["회계년도"] == "2014")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "대한그린파워(주)") & (df["회계년도"] == "2016")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "(주)알테오젠") & (df["회계년도"] == "2015")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "대한그린파워(주)") & (df["회계년도"] == "2016")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "(주)에이프로젠헬스케어") & (df["회계년도"] == "2017")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "한국전자금융(주)") & (df["회계년도"] == "2017")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "(주)아난티") & (df["회계년도"] == "2015")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "(주)헥토이노베이션") & (df["회계년도"] == "2016")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "(주)제이테크놀로지") & (df["회계년도"] == "2012")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "(주)상지카일룸") & (df["회계년도"] == "2017")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "(주)동원개발") & (df["회계년도"] == "2016")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "(주)동서") & (df["회계년도"] == "2015")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "(주)웹젠") & (df["회계년도"] == "2015")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "(주)지에스홈쇼핑") & (df["회계년도"] == "2015")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "(주)위메이드") & (df["회계년도"] == "2013")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "(주)케이피엠테크") & (df["회계년도"] == "2012")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "아이이(주)") & (df["회계년도"] == "2013")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "(주)아이디스홀딩스") & (df["회계년도"] == "2013")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "(주)와이비엠넷") & (df["회계년도"] == "2013")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "크리스탈지노믹스(주)") & (df["회계년도"] == "2013")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "아이진(주)") & (df["회계년도"] == "2016")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "(주)파라다이스") & (df["회계년도"] == "2014")].index , axis = 0 , inplace = True)

# df.drop(index = df[(df["회사명"]== "(주)셀트리온") & (df["회계년도"] == "2014")].index , axis = 0 , inplace = True)


# # 제거 후 인덱스 초기화
# df.reset_index(drop=True, inplace=True)



In [560]:
#  산업군별로 표준편차 확인하여 이상치 선별 및 제거

drop_data = [("(주)베노홀딩스", "2017"), ("대한그린파워(주)", "2015"), ("(주)레고켐바이오사이언스", "2014"),
             ("(주)스튜디오산타클로스엔터테인먼트", "2016"), ("(주)희림종합건축사사무소", "2017"), ("(주)알테오젠", "2015"),
             ("(주)엔케이맥스", "2016"), ("(주)정상제이엘에스", "2016"), ("(주)휴먼엔", "2017"),
             ("대한그린파워(주)", "2014"), ("대한그린파워(주)", "2016"), ("(주)에이프로젠헬스케어", "2017"),
             ("한국전자금융(주)", "2017"), ("(주)아난티", "2015"), ("(주)헥토이노베이션", "2016"),
             ("대한그린파워(주)", "2015"), ("(주)제이테크놀로지", "2012"), ("(주)상지카일룸", "2017"),
             ("(주)동원개발", "2016"), ("(주)동서", "2015"), ("(주)웹젠", "2015"),
             ("(주)지에스홈쇼핑", "2015"), ("(주)위메이드", "2013"), ("(주)케이피엠테크", "2012"),
             ("아이이(주)", "2013"), ("(주)아이디스홀딩스", "2013"), ("(주)와이비엠넷", "2013"),
             ("크리스탈지노믹스(주)", "2013"), ("아이진(주)", "2016"), ("(주)파라다이스", "2014"), ("(주)셀트리온", "2014")]

for data in drop_data:
    df.drop(index=df[(df["회사명"] == data[0]) & (df["회계년도"] == data[1])].index, axis=0, inplace=True)
    
    
# 제거 후 인덱스 초기화
df.reset_index(drop=True, inplace=True)

In [561]:
print('전체 행:', df.shape[0],'개')
print('전체 열:', df.shape[1],'개')
print('전체 기업 수',len(df['회사명'].unique()))
print("결측치 수 : ", df.isna().sum().sum())

전체 행: 9574 개
전체 열: 45 개
전체 기업 수 1440
결측치 수 :  0


### ESD이상치

In [562]:
# ### ESD이상치


# def EDS_outlier(df=None, col=None,weight=1.5):
#     # import numpy as np
#     m = np.mean(df[col])
#     sd = np.std(df[col])
#     sd_weight = sd*weight
    
#     lowest = m -sd_weight
#     highest = m + sd_weight

#     outlier_idx = df[col][ (df[col] < lowest) | (df[col] > highest) ].index
#     return outlier_idx


# for i in df.select_dtypes("float").columns:
#     oulier = EDS_outlier(df=df[df["target_3"]==0],col= i,weight=3)
#     df.drop(index= oulier, axis=0, inplace=True)
#     df.reset_index(drop=True, inplace=True)

# print('전체 행:', df.shape[0],'개')
# print('전체 열:', df.shape[1],'개')
# print('전체 기업 수',len(df['회사명'].unique()))
# print("결측치 수 : ", df.isna().sum().sum())

In [563]:
col_int = df.select_dtypes("float").columns
df_1 = df[df["target_3"]==1]

lower_percentile = 0.005
upper_percentile = 0.995

lower_cutoff = df[df['target_3'] == 0][col_int].quantile(lower_percentile)
upper_cutoff = df[df['target_3'] == 0][col_int].quantile(upper_percentile)

is_outlier = (df[df['target_3'] == 0][col_int] < lower_cutoff) | (df[df['target_3'] == 0][col_int] > upper_cutoff)

df_t = df[df['target_3'] == 0][~is_outlier.any(axis=1)]

print('전체 행:', df_t.shape[0],'개')
print('전체 열:', df_t.shape[1],'개')
print('전체 기업 수',len(df_t['회사명'].unique()))
print("결측치 수 : ", df_t.isna().sum().sum())

전체 행: 7669 개
전체 열: 45 개
전체 기업 수 1271
결측치 수 :  0


In [564]:
df = pd.concat([df_t, df_1], axis = 0 )
df.reset_index(drop=True, inplace = True)

In [565]:
print('전체 행:', df.shape[0],'개')
print('전체 열:', df.shape[1],'개')
print('전체 기업 수',len(df['회사명'].unique()))
print("결측치 수 : ", df.isna().sum().sum())

전체 행: 7906 개
전체 열: 45 개
전체 기업 수 1361
결측치 수 :  0


---
### 이상치 처리 후 qqplot + boxplot

In [566]:
# 데이터 타입에 따라 데이터 분리

df_float = df.select_dtypes("float")
df_target = df.select_dtypes("int")
df_obj = df.select_dtypes("object")

In [567]:
df_float.describe()

,차입금의존도,부채비율,순운전자본비율,총자본정상영업이익률,자기자본순이익률,총자본순이익률,매출액총이익률,금융비용부담률,자본금회전률,유형자산회전율,재고자산회전률,자기자본회전률,타인자본회전률,총자본회전률,외국인지분율,대주주지분율,유형자산증가율,매출액증가율,자기자본증가율,총자본증가율,정상영업이익증가율,누적수익성비율,이자부담률,부채상환계수,현금흐름 대 자산,현금흐름 대 매출액,TMD,시총/총자산,EBIT/총자산,당기전기영업손익,FINDEP,EBITDA마진율,PBR변화율,ROA변화율,현금흐름/총부채비율,영업현금흐름-단기차입금,abs(영업현금흐름-당기순이익)/매출액
count,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000,7906.000000
mean,22.130014,109.316814,19.407564,2.561653,-4.368333,-0.116164,22.207277,1.386346,18.680847,8.518779,50.841450,1.685791,2.495732,0.842751,4.387149,36.511934,17.646639,8.644303,10.821590,9.621531,-21.612176,0.143399,88.866384,48.496699,0.037928,0.030084,0.857390,1.079107,0.009232,7051.908614,-580.528730,4.541940,19.673108,-51.038064,0.158618,-11185.110928,0.144032
std,18.997717,768.751009,29.102477,8.661054,86.280431,12.701879,17.343327,4.997044,21.867153,23.575824,274.696179,1.373752,1.691216,0.457931,7.424174,15.879744,90.727268,36.330599,88.440620,25.903845,652.110343,0.727485,380.691731,308.014105,0.103177,0.269508,1.428415,1.088120,0.166774,16220.868594,19669.180539,21.263821,534.625715,600.720247,0.347096,36172.760543,0.406103
min,0.000000,0.000000,-1243.210000,-204.740000,-5300.390000,-288.540000,-207.540000,-12.900000,0.030000,0.130000,0.000000,0.000000,0.020000,0.020000,0.000000,0.210000,-100.000000,-98.900616,-1068.181818,-95.273372,-46068.571429,-38.243959,-724.306667,-363.390556,-3.752854,-13.633222,0.007277,0.014262,-4.745856,-166587.000000,-315346.856000,-951.933976,-10065.643303,-30791.304348,-2.906667,-830735.000000,0.000562
25%,7.532500,34.280000,3.410000,-0.620000,-5.365000,-2.860000,10.852500,-0.150000,5.512500,1.620000,4.330000,0.800000,1.280000,0.520000,0.670000,24.580000,-4.908716,-8.748995,-1.274681,-2.544723,-56.449970,0.015343,0.999823,0.073234,-0.008315,-0.011716,0.217839,0.462455,-0.017365,-503.500000,-6248.264750,1.474776,-20.858173,-72.395599,-0.019691,-17880.000000,0.029523
50%,20.200000,68.530000,20.285000,2.990000,3.750000,2.120000,18.805000,0.510000,11.870000,2.860000,7.365000,1.300000,2.050000,0.760000,1.770000,35.650000,1.275442,4.506066,5.975423,5.360820,2.075091,0.238934,5.803920,0.348240,0.043526,0.054784,0.473805,0.767612,0.030288,3239.000000,-190.530000,5.720238,-0.349379,-8.586205,0.107456,-4260.000000,0.068021
75%,34.090000,124.430000,36.817500,7.100000,9.890000,6.097500,31.187500,1.640000,23.370000,5.550000,14.140000,2.160000,3.250000,1.090000,4.350000,47.580000,16.039202,18.401089,15.878716,16.224426,65.323485,0.440663,27.495458,1.652267,0.092718,0.122759,0.999687,1.318904,0.071237,9675.250000,5775.287750,11.741460,28.005656,71.339429,0.285301,3805.750000,0.135573
max,749.400000,65433.660000,81.040000,30.530000,109.320000,129.020000,88.580000,184.340000,179.920000,444.060000,5730.060000,36.170000,10.570000,3.180000,59.630000,100.000000,3230.612245,423.783784,5045.605307,342.558976,2914.285714,0.936660,6408.164167,5667.640000,1.180479,1.298200,51.543305,33.247670,3.066083,150868.000000,263527.267000,51.783458,45142.470366,2671.428571,2.002052,136071.000000,16.775964


In [568]:
# qqplot(df_float,9,9)

In [569]:
# dist(df_float,df['target_3'],9,9)
# plt.show()

---
### 박스플롯을 보고 데이터 분포에 따라 상하단 위저를 주는 커럼과 상단에만 위져을 주는 컬럼 분리

In [570]:
# 상단_ls = ["자본", "유보율", "현금비율", "부채비율", "당좌비율", "감가상각비","자본금회전률", "유형자산회전율", 
# "재고자산회전률", "자기자본회전률", "타인자본회전률", "금융수익", "합계_기말인원(명)", "외국인지분율", 
# "단기차입금", "유동성장기부채" ,"CAPEX", "유형자산증가율", "매출액증가율", "총자본증가율","토빈Q", "TMD", 
# "시총/총자산", "연구개발비" , "RD", "직원수변동률%", "직원인건비총액", ]


# 상하단_ls=["차입금의존도","순운전자본비율", "영업손익","EBITDA", "총자본정상영업이익률","자기자본순이익률", "매출액총이익률",
# "금융비용부담률", "경영자본순이익률", "총자본회전률", "이익잉여금", "영업활동으로 인한 현금흐름", "법인세비용차감전순이익"
# ,"당기순이익", "대주주지분율", "자기자본증가율","정상영업이익증가율" ,"장단기금리차", "cpi", "누적수익성비율"
# ,"이자부담률", "부채상환계수", "현금흐름 대 자산" ,"현금흐름 대 매출액", "EBIT/총자산", "PER", "PBR", "전기당기영업손익"
# , "FINDEF", "PBR변화율", "ROA변화율", "현금흐름/총부채비율", "영업현금흐름-단기차입급" , '총자본순이익률',
# "(영업현금흐름 - 당기순이익)(절대값) / 매출액","로그자산", "로그이자비용", "로그매출액", "로그부채", "로그금융원가", "로그시가총액" ]

In [571]:
# 나눈 컬럼과 나누기 전 컬럼 차이가 있는지 확인

# 비교 =  (상단_ls + 상하단_ls)
# 전체 = df_float.columns


# def find_difference(list1, list2):
#     difference = []
#     for item in list1:
#         if item not in list2:
#             difference.append(item)
#     return difference

# find_difference(전체,비교)


In [572]:
# # 정규성이 강한 컬럼은 극단값 0.005 약한 컬럼은 극단값 0.01을 기준으로 윈저라이징

# df_win = pd.DataFrame()
# # 상단
# for i in 상단_ls:
#     a = winsorize(df_float[i],limits=[0.0, 0.05])
#     df_win[i] = a

# # 상하단
# for i in 상하단_ls:
#     a = winsorize(df_float[i],limits=[0.025, 0.025])
#     df_win[i] = a

# df_win.isna().sum().sum()

In [573]:
# # 컬럼 삽입
# for i in df_win.columns:
#     df_float[i] = df_win[i]

In [574]:
# qqplot(df_float,9,9)

In [575]:
# dist(df_float,df['target_3'],6,12)
# plt.show()

In [576]:
# train, test 분리

train = df[df["회계년도"] < 2019]
test = df[df["회계년도"] >= 2019]

In [577]:
# train, test shape 

print(train.shape)
print("train 기업수",len(train["회사명"].unique()))
print("타겟1:",len(train[train["target_1"] == 0]["회사명"].unique()), len(train[train["target_1"] == 1]["회사명"].unique()))
print("타겟2:",len(train[train["target_2"] == 0]["회사명"].unique()), len(train[train["target_2"] == 1]["회사명"].unique()))
print("타겟3:",len(train[train["target_3"] == 0]["회사명"].unique()), len(train[train["target_3"] == 1]["회사명"].unique()))
print(len(train[train["target_1"]==0]))
print(len(train[train["target_2"]==0]))
print(len(train[train["target_3"]==0]))
print("-----------")

print(test.shape)
print("test 기업수",len(test["회사명"].unique()))
print("타겟1:",len(test[test["target_1"] == 0]["회사명"].unique()), len(test[test["target_1"] == 1]["회사명"].unique()))
print("타겟2:",len(test[test["target_2"] == 0]["회사명"].unique()), len(test[test["target_2"] == 1]["회사명"].unique()))
print("타겟3:",len(test[test["target_3"] == 0]["회사명"].unique()), len(test[test["target_3"] == 1]["회사명"].unique()))
print(len(test[test["target_1"]==0]))
print(len(test[test["target_2"]==0]))
print(len(test[test["target_3"]==0]))
print(test["target_1"].sum())
print(test["target_2"].sum())
print(test["target_3"].sum())

(5014, 45)
train 기업수 1093
타겟1: 1023 110
타겟2: 984 113
타겟3: 1030 107
4842
4803
4849
-----------
(2892, 45)
test 기업수 1192
타겟1: 1138 55
타겟2: 1140 55
타겟3: 1141 55
2817
2819
2820
75
73
72


In [578]:
test['target_3'].value_counts(property)

0    0.975104
1    0.024896
Name: target_3, dtype: float64

In [579]:
#  거래소코드, 회계년도 , 상장일 데이터 타입 변환(qqplot 코드 만들때 편의를 위해)
df = train
df['거래소코드'] =df['거래소코드'].astype(str).str.rjust(6,'0')
df['회계년도'] = df['회계년도'].astype(int).astype(str)



test['거래소코드'] =test['거래소코드'].astype(str).str.rjust(6,'0')
test['회계년도'] = test['회계년도'].astype(int).astype(str)


In [580]:
# train, test 분리 후 인덱스 정리
df.reset_index(drop= True, inplace=True)
test.reset_index(drop=True, inplace=True)


In [583]:
test

,회사명,거래소코드,회계년도,차입금의존도,부채비율,순운전자본비율,산업군,총자본정상영업이익률,자기자본순이익률,총자본순이익률,매출액총이익률,금융비용부담률,자본금회전률,유형자산회전율,재고자산회전률,자기자본회전률,타인자본회전률,총자본회전률,외국인지분율,대주주지분율,유형자산증가율,매출액증가율,자기자본증가율,총자본증가율,정상영업이익증가율,업력,누적수익성비율,이자부담률,부채상환계수,현금흐름 대 자산,현금흐름 대 매출액,TMD,시총/총자산,EBIT/총자산,당기전기영업손익,FINDEP,EBITDA마진율,PBR변화율,ROA변화율,target_1,target_2,target_3,현금흐름/총부채비율,영업현금흐름-단기차입금,abs(영업현금흐름-당기순이익)/매출액
0,(주)CMG제약,058820,2019,0.15,6.71,73.49,제조업,1.70,2.15,1.99,49.34,-1.86,0.95,2.83,6.65,0.41,5.21,0.38,2.12,25.84,39.666454,17.563421,73.258833,68.073964,1.401869,18,0.026295,523.273333,523.273333,0.014685,0.048155,0.020473,3.070552,0.010050,2586.0,5352.812,8.033468,-26.117579,-63.284133,0,0,0,0.233595,2823.0,0.003889
1,(주)CMG제약,058820,2020,0.25,7.85,67.74,제조업,0.94,0.12,0.11,48.42,-1.27,0.93,2.39,5.35,0.36,4.90,0.33,7.20,25.84,23.309424,9.956843,0.540081,1.619305,-30.030722,19,0.028860,185.125652,185.125652,0.015070,0.045672,0.017719,4.109406,-0.000712,2213.0,4045.506,6.605476,35.269435,-94.472362,0,0,0,0.206960,2944.0,0.042336
2,(주)CMG제약,058820,2021,0.26,7.84,56.11,제조업,0.84,1.73,1.60,48.15,-2.54,1.00,2.31,4.92,0.38,4.84,0.35,6.62,25.84,2.605830,7.780019,1.971501,1.954893,-8.891328,20,0.044137,197.710000,197.710000,0.028262,0.081022,0.027170,2.674290,-0.002566,1741.0,-3352.742,5.976121,-34.933310,1354.545455,0,0,0,0.388958,5629.0,0.035466
3,(주)가비아,079940,2019,17.58,48.13,21.90,정보통신업,13.51,15.73,12.01,48.99,0.03,21.91,2.21,64.12,1.19,2.90,0.84,16.57,24.89,73.824506,15.209589,18.191213,32.009165,30.509127,20,0.353338,17.541068,1.656577,0.184833,0.249152,0.579583,0.560613,0.121181,20962.5,-12495.173,8.196484,10.150896,11.513463,0,0,0,0.568853,30306.0,0.106866
4,(주)가비아,079940,2020,19.38,50.13,28.10,정보통신업,12.41,14.99,10.33,49.20,-0.29,24.98,1.77,59.27,1.10,2.23,0.74,12.07,24.85,23.214620,13.983157,28.338236,30.067128,20.255298,21,0.316008,28.844333,1.545856,0.192608,0.296270,0.487780,0.684527,0.104086,26141.0,-2588.479,8.309734,23.748331,-13.988343,0,0,0,0.576847,41483.0,0.155763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2887,에이아이비트(주),039230,2019,2.97,34.87,50.11,제조업,4.41,-28.25,-15.68,38.57,6.06,0.93,7.53,14.70,0.95,1.18,0.52,0.56,5.41,998.673100,-0.971722,76.042430,-8.802874,19.242424,23,-1.227368,0.338720,0.338720,-0.043766,-0.079624,0.425672,0.607366,0.013873,2894.0,3566.661,2.228791,-65.471420,-7.988981,1,1,1,-0.169281,-2978.0,0.219406
2888,에이아이비트(주),039230,2020,5.70,28.41,41.17,제조업,-5.84,-16.41,-12.46,33.63,2.66,0.54,3.22,10.54,0.61,1.92,0.46,0.34,5.12,-0.966184,-21.895136,-10.302862,-14.596438,-216.994917,24,-1.572291,-4.033239,-3.917877,0.019118,0.038032,0.643634,0.343744,-0.113711,-267.5,-862.761,-12.191394,-46.113268,20.535714,1,1,1,0.086412,1085.0,0.307031
2889,이엘케이(주),094190,2019,134.43,0.00,-136.56,제조업,-57.27,0.00,-70.86,-81.58,3.25,1.40,1.24,2.17,0.00,0.47,0.55,0.00,5.15,-40.796795,-74.491540,-362.043564,-56.180086,-216.882171,20,-4.030803,-31.312486,-0.898748,0.040668,0.045175,1.844485,1.024756,-1.129080,-34336.5,-1318.815,-103.007779,-131.008545,-234.087694,1,1,1,0.021516,-48648.0,1.336826
2890,지나인제약(주),078650,2021,56.40,0.00,-38.76,제조업,-33.23,-517.98,-56.79,-59.13,5.89,1.30,0.63,6.51,0.00,0.41,0.36,0.76,6.32,-35.810443,-42.651200,-130.493815,-31.802274,31.618408,22,-1.767853,-6.300376,-0.405769,-0.107389,-0.239750,2.316560,0.482884,-0.572906,-26837.5,11203.679,-38.681648,-221.527505,-9.316651,1,1,1,-0.096001,-25867.0,1.323765


In [582]:
df.to_csv("./datasets/전처리준비/코스닥_train_이상치처리완료.csv", encoding="utf-8-sig")
test.to_csv("./datasets/전처리준비/코스닥_test_이상치처리완료.csv", encoding="utf-8-sig")